In [2]:

#Importing all the required libraries

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

import numpy as np

import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk

In [2]:
#Setting up the directories
train_dir = r"D:/data/train"
valid_dir = r"D:/data/valid"
test_dir = r"D:/data/test"

# Set up ImageDataGenerators for loading images
train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Load images from directories
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(64, 64), batch_size=32, class_mode='binary')
valid_generator = valid_datagen.flow_from_directory(valid_dir, target_size=(64, 64), batch_size=32, class_mode='binary')
test_generator = test_datagen.flow_from_directory(test_dir, target_size=(64, 64), batch_size=32, class_mode='binary')

Found 30250 images belonging to 2 classes.
Found 6300 images belonging to 2 classes.
Found 6300 images belonging to 2 classes.


In [4]:
# Building a simple CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification: wildfire or no wildfire
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


C:\Users\HP\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [5]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True


In [5]:
# Train the model
history = model.fit(train_generator, validation_data=valid_generator, epochs=10, verbose=1)


C:\Users\HP\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
946/946 ━━━━━━━━━━━━━━━━━━━━ 954s 1s/step - accuracy: 0.8663 - loss: 0.3028 - val_accuracy: 0.9359 - val_loss: 0.1715
Epoch 2/10
946/946 ━━━━━━━━━━━━━━━━━━━━ 878s 928ms/step - accuracy: 0.9308 - loss: 0.1894 - val_accuracy: 0.9506 - val_loss: 0.1294
Epoch 3/10
946/946 ━━━━━━━━━━━━━━━━━━━━ 797s 842ms/step - accuracy: 0.9395 - loss: 0.1654 - val_accuracy: 0.9521 - val_loss: 0.1425
Epoch 4/10
946/946 ━━━━━━━━━━━━━━━━━━━━ 535s 564ms/step - accuracy: 0.9480 - loss: 0.1427 - val_accuracy: 0.9576 - val_loss: 0.1158
Epoch 5/10
946/946 ━━━━━━━━━━━━━━━━━━━━ 429s 453ms/step - accuracy: 0.9499 - loss: 0.1319 - val_accuracy: 0.9476 - val_loss: 0.1424
Epoch 6/10
946/946 ━━━━━━━━━━━━━━━━━━━━ 471s 498ms/step - accuracy: 0.9533 - loss: 0.1261 - val_accuracy: 0.9571 - val_loss: 0.1172
Epoch 7/10
946/946 ━━━━━━━━━━━━━━━━━━━━ 493s 522ms/step - accuracy: 0.9610 - loss: 0.1063 - val_accuracy: 0.9546 - val_loss: 0.1207
Epoch 8/10
946/946 ━━━━━━━━━━━━━━━━━━━━ 423s 447ms/step - accuracy: 0.9664 - lo

In [6]:
model.save("wildfire_model.h5")

In [7]:
# Function to load and predict an image
def predict_image():
    # Open file dialog to select an image
    file_path = filedialog.askopenfilename()
    if file_path:
        # Display the image in the GUI
        img = Image.open(file_path)
        img = img.resize((200, 200))
        img = ImageTk.PhotoImage(img)
        image_label.configure(image=img)
        image_label.image = img

        # Preprocess the image for the model
        img_for_model = Image.open(file_path).resize((64, 64))
        img_array = np.array(img_for_model) / 255.0  # Rescale like during training
        img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

        # Make a prediction
        prediction = model.predict(img_array)[0][0]
        result = "Wildfire" if prediction > 0.5 else "No Wildfire"
        result_label.config(text="Prediction: " + result)

# Setting up the GUI window
root = tk.Tk()
root.title("Forest Fire Detection")
root.geometry("400x400")

# Add widgets
btn = tk.Button(root, text="Upload Image", command=predict_image)
btn.pack(pady=20)

image_label = tk.Label(root)
image_label.pack()

result_label = tk.Label(root, text="Prediction: ", font=("Helvetica", 16))
result_label.pack(pady=20)

root.mainloop()
